#Pandas

##What is Pandas?
A Python library providing data structures and data analysis tools.

##Why
- Alternative to Excel or R
- Based on Data Frames (think of it like a table) and Series (single column table / time series)

##Learning Pandas
* Almost anything you want to do is already a built-in function in Pandas.
* Before you decide to write a function to do some kind of operation on a Pandas object, scour the Pandas docs and StackOverflow
* http://pandas.pydata.org/pandas-docs/stable/index.html

#Objectives

- Create/Understand Series objects
- Create/Understand DataFrame objects
- Create and destroy new columns, apply functions to rows and columns
- Join/Merge Dataframes
- Use DataFrame grouping and aggregation
- Perform high-level EDA using Pandas

### Standard Imports

In [ ]:
# By convention import pandas like:
import pandas as pd
import numpy as np

# For fake data.
from numpy.random import randn

#Series

Think of a Pandas Series as a _labeled_ one-dimensional vector. In fact, it need not be a numeric vector, it can contain arbitrary python objects.

Integer valued series:

Real valued series:

String valued series:

#Indexes.

Notice how each series has an index (in this case a relatively meaningless default index). Pandas can make great use of informative indexes. Indexes work similarly to a dictionary key, allowing fast lookups of the data associated with the index---which helps optimize many operations.

In [ ]:
# Sample index - each data point is labelled with a state.
index1 = ['California', 'Alabama', 'Indiana', 'Montana', 'Kentucky']
index2 = ['Washington', 'Alabama', 'Montana', 'Indiana', 'New York']

Labelled numeric series:

In [ ]:
series1 = pd.Series(randn(5))
series2 = pd.Series(randn(5))
print series1, '\n'*2, series2

The index is used to line up arithmetic operations.

Aggregation by index labels is easy (and optimized)

In [ ]:
long_index = index1*3
long_series = pd.Series(randn(15), index=long_index)
print long_series

Create a series indexed by dates

Closer look at the index values

Resample by week

#DataFrames
Data frames extend the concept of Series to table-like data.

From a dictionary of series or list

In [ ]:
s1 = pd.Series(randn(10))
s2 = randn(10)

From a numpy array

Dataframes can be indexed (selected) by label, numeric index (avoid if possible), and boolean.

Using labels

Using indexes

Using booleans

Each column is a series

Each row is a series

Every column shares the same index

The index for each row is the column headers

#DataFrame basic operations

Load data from a delimited file.

In [ ]:
file_path = '../data/playgolf.csv'

Create a new column.

Drop a column

Delete a row

Add a row

#Applying functions

Using existing functions

Creating and using an arbitrary function

Using multiple values on each row

#Summarizing dataframe

In [ ]:
df.describe()

In [ ]:
df.info()

#Index manipulation

Set index

Reset index

#Grouping (split-apply-combine)

Get averages for each outlook

Initialize a groupby object---and iterate through the groupings

Get the mean for each group

Normalize by the group each date belonged in

Summarize by each group

#Cross Comparison

#Charting

In [ ]:
#this enables charts to show inside iPython Notebook
from matplotlib import pyplot as plt
%matplotlib inline

#Using pandas with sql

In [ ]:
import psycopg2 as pg2

In [ ]:
conn = pg2.connect(dbname='test', user='minghuang', host = 'localhost')

In [ ]:
query = '''
        SELECT *
        FROM logins
        LIMIT 30;
        '''

In [ ]:
conn.close()

#We're done with the Pandas tutorial, but.....

here are some examples showing how to redo the SQL assignment in Pandas, plus Exploratory Data Analysis using Pandas using the stuffs you just learnt (hopefully).

#ReadyChef
[Data](https://www.dropbox.com/sh/5sm9nvnh6b4m8d0/AABQyediVavAdsjnoEUBEyYCa?dl=0)

Download, unzip and place the readychef directory in pandas-tutorial/data

In [ ]:
meals = pd.read_csv('../data/readychef/meals.csv')
events = pd.read_csv('../data/readychef/events.csv')
referrals = pd.read_csv('../data/readychef/referrals.csv')
users = pd.read_csv('../data/readychef/users.csv')
visits = pd.read_csv('../data/readychef/visits.csv')

Select statements
===================

1. To get an understanding of the data, run a [SELECT](http://www.postgresqltutorial.com/postgresql-select/) statement on each table. Keep all the columns and limit the number of rows to 10.

2. Write a `SELECT` statement that would get just the userids.

3. Maybe you're just interested in what the campaign ids are. Use 'SELECT DISTINCT' to figure out all the possible values of that column.

    *Note:*  Pinterest=PI, Facebook=FB, Twitter=TW, and Reddit=RE

In [ ]:
#3
users.campaign_id.unique()
users.campaign_id.value_counts()

Where Clauses / Filtering
========================================

Now that we have the lay of the land, we're interested in the subset of users that came from Facebook (FB). If you're unfamiliar with SQL syntax, the [WHERE](http://www.postgresqltutorial.com/postgresql-where/) clause can be used to add a conditional to `SELECT` statements. This has the effect of only returning rows where the conditional evaluates to `TRUE`. 

*Note: Make sure you put string literals in single quotes, like `campaign_id='TW'`.*

1. Using the `WHERE` clause, write a new `SELECT` statement that returns all rows where `Campaign_ID` is equal to `FB`.

2. We don't need the campaign id in the result since they are all the same, so only include the other two columns.

    Your output should be something like this:

    ```
     userid |     dt
    --------+------------
          3 | 2013-01-01
          4 | 2013-01-01
          5 | 2013-01-01
          6 | 2013-01-01
          8 | 2013-01-01
    ...
    ```


In [ ]:
# can also use users.ix[users.campaign_id=='FB']
users[users['campaign_id']=='FB'][['userid', 'dt']].head(10)

Aggregation Functions
=======================



6. Now get the average price, the min price and the max price for each meal type. Don't forget the group by statement!

    Your output should look like this:

    ```
        type    |         avg         | min | max
    ------------+---------------------+-----+-----
     mexican    |  9.6975945017182131 |   6 |  13
     french     | 11.5420000000000000 |   7 |  16
     japanese   |  9.3804878048780488 |   6 |  13
     italian    | 11.2926136363636364 |   7 |  16
     chinese    |  9.5187165775401070 |   6 |  13
     vietnamese |  9.2830188679245283 |   6 |  13
    (6 rows)
    ```



In [ ]:
meals.groupby('type')['price'].aggregate({'avg': np.mean,
                                          'min': np.min,
                                          'max': np.max})

Joins
=========================

Now we are ready to do operations on multiple tables. A [JOIN](http://www.tutorialspoint.com/postgresql/postgresql_using_joins.htm) allows us to combine multiple tables.

1. Write a query to get one table that joins the `events` table with the `users` table (on `userid`) to create the following table.

    ```
     userid | campaign_id | meal_id | event
    --------+-------------+---------+--------
          3 | FB          |      18 | bought
          7 | PI          |       1 | like
         10 | TW          |      29 | bought
         11 | RE          |      19 | share
         15 | RE          |      33 | like
    ...
    ```



In [ ]:
events.merge(users, how='inner', left_on='userid', right_on='userid')[['userid', 'meal_id', 'campaign_id']].head(3)

Extra Credit
========================
1. Answer the question, _"What user from each campaign bought the most items?"_

    It will be helpful to create a temporary table that contains the counts of the number of items each user bought. You can create a table like this: `CREATE TABLE mytable AS SELECT...`

In [ ]:
# Get a table with user, campaign, and event
comb = users.merge(events, on='userid')[['userid', 'campaign_id', 'event']]
# Get the number of buys for each user/campaign combination
agg = comb.groupby(['userid', 'campaign_id'])['event'].aggregate({'num_buys': lambda x: sum(x=='bought')}).reset_index()
# Add a column of max number of buys per campaign
agg['max_buys'] = agg.groupby('campaign_id')['num_buys'].transform(np.max)
# Find the users with the maximum buys
agg.ix[agg.num_buys == agg.max_buys]

#Exploratory Data Analysis with Pandas

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('../data/playgolf.csv', delimiter='|' )
print df.head()

#Describe the continuous variables
##This treats the Boolean Windy variable as a series of 0's and 1's

In [ ]:
df.describe()

Can see the general pattern of Temperature and Humidity and mean of a Boolean represents the percentage

##We can make use of df.plot() to produce simple graphs that calls on the more adjustable Matplotlib library 

In [ ]:
# Side-by-side histograms
df[['Humidity', 'Temperature']].hist(bins=5)

In [ ]:
# Box plot
df[['Humidity', 'Temperature']].plot(kind='box')

###Scatterplots for examining bivariate relationships (kind=scatter)

In [ ]:
df.plot('Humidity', 'Temperature', kind='scatter')

###If we want to color the scatterplots according to a category, it requires a bit of matplotlib...ugh!

In [ ]:
grouped = df.groupby('Outlook')
for name, group in grouped:
    plt.plot(group.Humidity, group.Temperature, label=name, marker='o', linestyle='')
plt.legend(numpoints=1)
plt.show()

#What about the categorical variables? Frequency tables and relative frequency tables

###Simply df.value_counts() gets you the frequencies

In [ ]:
df.Outlook.value_counts()

###Using apply will get you the value counts for multiple columns at once

In [ ]:
df.head()

###Contingency Tables for looking at bivariate relationships between two categorical variables (pd.crosstab)

In [ ]:
pd.crosstab(df.Outlook, df.Result)

###Often we want the row percentages

In [ ]:
pd.crosstab(df.Outlook, df.Result).apply(lambda x: x/x.sum(), axis=0)

###Or the column percentages

In [ ]:
pd.crosstab(df.Outlook, df.Result).apply(lambda x: x/x.sum(), axis=1)

#Enough...lets get to the pair sprint